<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Import-data" data-toc-modified-id="Import-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import data</a></span></li><li><span><a href="#Create-config" data-toc-modified-id="Create-config-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Create config</a></span></li></ul></li><li><span><a href="#Model---Generic-Functions-(Relevant-for-all-scenarios)" data-toc-modified-id="Model---Generic-Functions-(Relevant-for-all-scenarios)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model - Generic Functions (Relevant for all scenarios)</a></span><ul class="toc-item"><li><span><a href="#Create-One-step-Transition-Matrix-for-Disease-Progression" data-toc-modified-id="Create-One-step-Transition-Matrix-for-Disease-Progression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Create One-step Transition Matrix for Disease Progression</a></span></li><li><span><a href="#Create_Pai0---Initial-Condition" data-toc-modified-id="Create_Pai0---Initial-Condition-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Create_Pai0 - Initial Condition</a></span></li><li><span><a href="#Life-Expectancy" data-toc-modified-id="Life-Expectancy-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Life Expectancy</a></span></li><li><span><a href="#Sum_checkups" data-toc-modified-id="Sum_checkups-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Sum_checkups</a></span></li><li><span><a href="#Screening-Costs" data-toc-modified-id="Screening-Costs-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Screening Costs</a></span></li><li><span><a href="#Calculate-Utility" data-toc-modified-id="Calculate-Utility-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Calculate Utility</a></span></li><li><span><a href="#Calculate-Country-Utility-if-no-Treatment-is-applied" data-toc-modified-id="Calculate-Country-Utility-if-no-Treatment-is-applied-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Calculate Country Utility if no Treatment is applied</a></span></li><li><span><a href="#Optimal-Policy" data-toc-modified-id="Optimal-Policy-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Optimal Policy</a></span></li></ul></li><li><span><a href="#Run-Scenarios" data-toc-modified-id="Run-Scenarios-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Run Scenarios</a></span><ul class="toc-item"><li><span><a href="#Basic-Model-+-One-time-Screening-campaign" data-toc-modified-id="Basic-Model-+-One-time-Screening-campaign-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Basic Model + One-time Screening campaign</a></span></li><li><span><a href="#Extension-2---Discount-for-Quantity" data-toc-modified-id="Extension-2---Discount-for-Quantity-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Extension 2 - Discount for Quantity</a></span></li><li><span><a href="#Extension-3---Timed-Screening-Following-Discount-for-Quantity" data-toc-modified-id="Extension-3---Timed-Screening-Following-Discount-for-Quantity-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Extension 3 - Timed Screening Following Discount for Quantity</a></span></li><li><span><a href="#Extension-4---Patent-Expiry" data-toc-modified-id="Extension-4---Patent-Expiry-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Extension 4 - Patent Expiry</a></span></li></ul></li></ul></div>

# Setup

In [6]:
import numpy as np 
import pandas as pd
import timeit
import itertools
from tqdm import tqdm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

## Import data

In [7]:
# Disease progression rates
male_progress_df = pd.read_csv('input_data_for_model/male progress by age.csv')
female_progress_df = pd.read_csv('input_data_for_model/female progress by age.csv')

# Mortality rates
female_mortality_df = pd.read_csv('input_data_for_model/female mortality rate.csv')
male_mortality_df = pd.read_csv('input_data_for_model/male mortality rate.csv')

# Initial conditions from simulation
female_pai0_df = pd.read_csv('input_data_for_model/2012female_simOct18.csv')
male_pai0_df = pd.read_csv('input_data_for_model/2012male_simOct18.csv')

# Population proportions & Infected proportion in population
pop_prop_df = pd.read_csv('input_data_for_model/2012popprop.csv')
inf_pop_prop_df = pd.read_csv('input_data_for_model/2012infpopprop.csv')

## Create config
Defining a dictionary containing all parameters in model

In [8]:
config =dict(
prop_diagnosed = 0.3 # Initial proportion of diagnosed
,gdp = 40000 # Israel GDP
    
#building matrix (Age,Stages)*(Age,Stages)
,num_stages = 4
,num_age_groups= 16
,num_absorption_stages=5   #0=Not_treated_HCV_death, 1=inefficient_treatment_HCV_death
                        #2=treated_non_HCV_death, 3=0=Not_treated_non_HCV_death, #4=Capitalization
,num_transient_stages=21
,genders =range(2)  # 0- Female, 1- Male)
,step=365 #the model is in resol0ution of days
,cost_efficient=np.array([0.015,0.002])
,s_screen=40 #data from ministry of health
,efficacy=np.array([0.98,0.98,0.92,0.79]) # efficacy of treatment per stage.
,s_m=330000 #costs due to mortality from HCV # costs of liver transplants
,s_c=160 #costs of checkups  
,total_s_t=list(range(0, 200000, 1000)) #Treatment costs

)

# Add to config more values that are calculated based on the already-defined keys in the dictionary
config['num_disease_stages']= (config['num_stages']*6)+1 #0=(F0,D-1), 4=(F0,D0), 19=(F3,D3), 20= F0 inefficient treatment, 24=Treated
config['num_transient_states'] = config['num_disease_stages'] - config['num_absorption_stages']
config['r'] = 0.03/config['step'] #discount rate - needs to be in units of days!
config['capital'] = 1/(1+config['r'])
config['alpha']=(1.0/5)/config['step'] #transition rate from one age group to another (except for the last one)
# config['v']=[1*config['gdp'],3*config['gdp']]  # financial value of life year (QALY)  #32500 GDP per capita in Israel 2012
config['v']=[1*config['gdp']]  # financial value of life year (QALY)  #32500 GDP per capita in Israel 2012
options=list(itertools.product(range(config['num_stages']), range(config['num_age_groups'])))
options.insert(0,((-1,-1))) #no treatment- baseline
config['policies'] = options

#############################
# all checkup options
config['yi_options'] = np.array([2,1,2/3,1/2,1/3,1/4,1/5])/config['step']  # amount of checkups in a year (possible y options)
# less checkup options for testing
# config['yi_options'] = np.array([1/3, 1/4])/config['step']  

yi_all=list(itertools.product(config['yi_options'],config['yi_options'],config['yi_options'],range(1)))
for i in range(len(yi_all)-1,-1,-1):
    if yi_all[i][0]>yi_all[i][1] or yi_all[i][1]>yi_all[i][2]:
        yi_all.pop(i)
config['yi_all']=yi_all
#############################
# gamma&yi array that will represents the diagnosis rates 
#(from undiagnosed to diagnosed in the first element and the checkups rates per stage afterwards)
# decision variables.
# yi depends only on the stage of the disease
config['gamma'] = 0.006/config['step']

config['optional_ps']= np.array(range(0,7500000,50000)) # Possible threshold values 
config['hcc_to_hcvdeath'] = 0.238

progress=np.append([female_progress_df.values],[male_progress_df.values], axis=0)
progress[:,:,:]=progress[:,:,:]/100
progress=progress.T
beta=np.full((16,7,2),0.0)
beta[:,:-1,:]=progress[:,:,:]
beta[:,-1,:]=(progress[:,3,:]*progress[:,4,:]+progress[:,5,:])*config['hcc_to_hcvdeath']
beta[:,:,:]=beta[:,:,:]/config['step']
# 0-age, 1-stagetostage, 2-gender
# last row of beta is moving to HCV death. Mu is the regular mortality rate
config['beta'] = beta

# taking the last row in the mortality df (We will use the mortality rates which belong to the last year in the data)
Mu = np.append([female_mortality_df.iloc[-1,:].values],[male_mortality_df.iloc[-1,:].values], axis=0)
Mu[:,:]=Mu[:,:]/config['step']
config['Mu']=Mu

config['gl_pai'] = np.append([female_pai0_df.values],[male_pai0_df.values], axis=0)

config['pop_prop'] = np.array(pop_prop_df) #population proportion - per gender & age group
config['inf_pop_prop'] = np.array(inf_pop_prop_df) #population proportion infected - per gender & age group

 **<font color=blue>Initialize the stages of the model</font>**

In [9]:
stages =  [(0,-1),(1,-1),(2,-1),(3,-1),(0,0),(1,0),\
    (1,1),(2,0),(2,1),\
    (2,2),(3,0),(3,1),(3,2),\
    (3,3),\
    ('u0'),('u1'),('u2'),('u3'),\
    ('success'),\
    ('m1'),('m2'),('m3'),('m4'),\
    ('capital')]

transient_stages =  stages[:-(config['num_absorption_stages']):] # all stages except the last 5 absorbing ones
absorbing_stages =  stages[:-(config['num_absorption_stages'])-1:-1][::-1] #Taking the last 5 stages, then resorting them so the order would be m1,m2,m3...

# Create list of indices to work on QR0I matrix
transient_indices_dict = {}
ind = 0
mult=1
mult_2=config['num_age_groups']
for stage in transient_stages:
        if stage != 'capital':
            transient_indices_dict[stage]=list(range((mult-1)*mult_2,(mult)*16))
            mult+=1
        else:
            transient_indices_dict[stage]=[(mult-1)*mult_2]

absorbing_indices_dict = {}
counter = (mult-1)*config['num_age_groups'] # initialize counter to start right after the end of the transient indices dict
for stage in absorbing_stages:
            absorbing_indices_dict[stage]=counter
            counter+=1
           
            
# Indices for all states in the matrix 
indices_dict = {**transient_indices_dict, **absorbing_indices_dict}

# Rewrite the absorbing state indices so they will start at 0 (for the r matrix)
new_val=0
for k in absorbing_indices_dict:
    absorbing_indices_dict[k]=new_val
    new_val+=1

# Model - Generic Functions (Relevant for all scenarios)

## Create One-step Transition Matrix for Disease Progression
1. Creating dictionaries & lists to place stages relevant for each type of transition

In [10]:
# Get indices of groups of states
#   1. f=d
#   2. d=-1
#   3. f=3
#   4. Treated population
#   5. f state
#   6. d state

# f=d
states_where_f_equals_d = []
for key in transient_indices_dict.keys():
    if key[0]==key[1]:
        states_where_f_equals_d.append(key)
        
states_where_f_equals_d_indices = {key:indices_dict[key] for key in states_where_f_equals_d if k in indices_dict}

# d=-1
states_not_discovered = []
for key in transient_indices_dict.keys():
    if key[1]==-1:
        states_not_discovered.append(key)
states_not_discovered_indices = {key:indices_dict[key] for key in states_not_discovered if k in indices_dict}
states_not_discovered_indices

# f=3
states_f_equals_3 = []
for key in transient_indices_dict.keys():
    if key[0]==3:
        states_f_equals_3.append(key)

states_f_equals_3.append('u3')
states_f_equals_3_indices = {key:indices_dict[key] for key in states_f_equals_3 if k in indices_dict}
states_f_equals_3_indices

# Treated population - we'll identify them by the fact that the keys in the dictionary are string and not integers in a tuple
states_treated = []
for key in transient_indices_dict.keys():
    if isinstance(key, str):
        states_treated.append(key)

treated_indices = {key:indices_dict[key] for key in states_treated if k in indices_dict}


# Understand f state per stage
f_state_per_stage={}
for stage in transient_indices_dict:
    if isinstance(stage,tuple): # if the state is a tuple then it looks like (f,d)
        f_state_per_stage[stage]=stage[0]
    else: # The state is a string, which means it's u0/u1/u2/u3 or "success"
        if stage[0]=='u':
                f_state_per_stage[stage]=int(stage[-1])           

# Understand d state per stage
d_state_per_stage={}
for stage in transient_indices_dict:
    if isinstance(stage,tuple): # if the state is a tuple then it looks like (f,d)
        d_state_per_stage[stage]=stage[1]
                
# Build dictionary that matches for every untreated state their untreated-discovered state
untreated_corresponding_discovered = {}
for key in transient_indices_dict.keys():
    if isinstance(key,tuple):
        if key[0]!=key[1] and key[1]!=-1:
            untreated_corresponding_discovered[key]=(key[0],key[0])

# Build dictionary that matches for state, its disease progression state
progression_states = {}
for stage in transient_indices_dict.keys():
    if isinstance(stage,tuple):
        if stage[0]!=3:
            progression_states[stage]=(stage[0]+1,stage[1])
    else: # The state is a string, which means it's u0/u1/u2/u3 or "success"
        if stage[0]=='u' and stage[-1]!='3':
            current_state=stage[-1]
            progression_states[stage]='u'+str(int(current_state)+1) 

In [11]:
# Defining dictionaries for memoization (better runtime)
matrix_dict = {}
pai0_dict = {}
delta_scanning_dict = {}
utility_dict={}
utility_vars_dict={}

2. One-step Transition Matrix

In [ ]:
pd.DataFrame()

In [23]:
# Save Create_matrix results to improve run time
def create_matrix(dicti,treatment_policy,gender, yi):
    """qxq one step transition Matrix"""
    # We save each calculation of this function in a dictionary - "matrix_dict". If we've already calculated then use the saved results, else calculate from scratch
    key = (treatment_policy,gender,yi)
    
    if key in matrix_dict:
        return matrix_dict[key]
    
    else:
        num_age_groups = dicti['num_age_groups']
        num_disease_stages = dicti['num_disease_stages']
        num_stages = dicti['num_stages']
        num_absorption_stages = dicti['num_absorption_stages']
        capital = dicti['capital']
        alpha = dicti['alpha']
        efficacy = dicti['efficacy']
        gamma = dicti['gamma']
        beta = dicti['beta']
        Mu = dicti['Mu']


        q_row_size = len(transient_indices_dict)*num_age_groups
        q = np.zeros((q_row_size,q_row_size))
        r = np.zeros((q_row_size,num_absorption_stages))
        for stage in transient_indices_dict:
            for row in indices_dict[stage]:  
                age = indices_dict[stage].index(row)# get the age group of the current row


                if row!=max(transient_indices_dict[stage]): # If not in last age group (last row for each stage) - advance in age group:
                    q[row,row+1] = alpha*capital

                if stage != 'success': # The following is not relevant for success so ignoring it
                    f_state = f_state_per_stage[stage] # Understanding the fibrosis state of the stage

                    if stage in progression_states.keys(): #only stages that are not in the last stage of the disease - the disease can progress
                        corresponding_progression_state = progression_states[stage]
                        q[row,transient_indices_dict[corresponding_progression_state][age]] = beta[age,f_state,gender]*capital

                    if stage in states_not_discovered: # if not discovered state, can transition to the discovered state where f=d
                        discovered_stage = (stage[0],stage[0])
                        q[row,transient_indices_dict[discovered_stage][age]] = gamma*capital

                    else: # all other stages that their d!=-1
                        # If stage is in policy
                        d_state = stage[1] 
                        if stage in d_state_per_stage:
                            if (d_state>=treatment_policy[0] and age<=treatment_policy[1]) or (treatment_policy[0]>=0 and d_state>treatment_policy[0]): #We will transition to Treated - either to success/corresponding unsuccessful. The second condition (fstate>policy) is relevant for cases where f_state is in policy but age is not, e.g. if policy is (2,2) and we're in f_state 3 and age 1.
                                q[row,transient_indices_dict['success'][age]] = yi[d_state]*efficacy[f_state]*capital  #if diagnosed and treated successfuly --> go to success state
                                
                                #find the corresponding stage in the Untreated group, e.g. 'u3' corresponds to all (f=3,d=*) states
                                unsuccesfully_treated_corresponding_stage = ('u'+str(f_state))
                                q[row,transient_indices_dict[unsuccesfully_treated_corresponding_stage][age]] = yi[d_state]*(1-efficacy[f_state])*capital  #if diagnosed and treated unsuccessfuly --> go to the corresponding unsuccessful state                

 
                        else: #stage is not in policy
                            if stage in untreated_corresponding_discovered.keys(): #the stages that are not undiscovered (d>=0) but f<>d [e.g. (1,0),(2,0),(2,1)...]
                                discovered_stage = (stage[0],stage[0])
                                q[row,transient_indices_dict[discovered_stage][age]] = yi[d_state]*capital  #stage diagnosis (frequent checkups)                

            #             else: #if the stage is *NOT* in policy
            #                 q[row,transient_indices_dict[discovered_stage][age]] = yi[f_state]*capital  #if diagnosed and treated unsuccessfuly --> go to the corresponding unsuccessful state

        ########
               #creating R
                if stage in states_treated: # Treated population         
                    r[row,absorbing_indices_dict['m3']]=Mu[gender,age]*capital # Treated non-HCV death

                    if stage in states_f_equals_3: # Treated population & fibrosis state equals 3
                        r[row,absorbing_indices_dict['m4']]=beta[age,-1,gender]*capital # Treated HCV death

                else: # Untreated population
                    r[row,absorbing_indices_dict['m1']]=Mu[gender,age]*capital # Untreated non-HCV death 
                    if stage in states_f_equals_3:
                        r[row,absorbing_indices_dict['m2']]=beta[age,-1,gender]*capital # Untreated HCV death

                # to capitalization
                r[row,absorbing_indices_dict['capital']]=1-capital

                # staying in the same stage
                q[row,row]=1-q[row].sum()-r[row].sum()            
        #############
        matrix_dict[treatment_policy,gender, yi] = (q,r)
        
        return q,r

3. Time to Absorption matrix

In [24]:
def N_and_B(dicti,treatment_policy,gender, yi):
    """N - time to Absorption matrix from each state,
    B - % of getting absorbed in each absorbing state' """
    
    
    q,r= create_matrix(dicti,treatment_policy,gender, yi)
    N_minus=np.matrix(np.identity(len(q))-q)
    N=np.linalg.inv(N_minus)
    B=np.dot(N,r)

    return N,B

## Create_Pai0 - Initial Condition

In [25]:
pai_indices_dict={}
counter=0
for key in indices_dict:
    pai_indices_dict[key]=counter
    counter+=1

In [16]:
def delta_scanning(dicti,screening_pop):
    '''Input: Number of people the government wants to scan
       Output: The percentage of people detected'''
        # We save each calculation of this function in a dictionary - "matrix_dict". If we've already calculated then use the saved results, else calculate from scratch
    key = (screening_pop)
    
    if key in delta_scanning_dict:
        return delta_scanning_dict[key]
    
    
    pai = dicti['gl_pai']
    pai_sum_female = np.sum(pai[0],axis=1)
    pai_sum_male = np.sum(pai[1],axis=1)
    pai_sum = np.array((pai_sum_female,pai_sum_male)).T
    prop_diagnosed = dicti['prop_diagnosed']
    inf_pop_prop = dicti['inf_pop_prop']
    already_diagnosed = prop_diagnosed*pai
    num_already_diagnosed = already_diagnosed.sum() # 30% of the population is already diagnosed
    not_diagnosed_yet = (1-prop_diagnosed)*pai
    
    proportion_of_sick_out_of_sick = pai/pai.sum()
    
    not_diagnosed_yet = (1-prop_diagnosed)*pai
    D = not_diagnosed_yet.sum() # Number of infected not diagnosed yet
    
    population_size = np.multiply(1/inf_pop_prop,pai_sum).sum()  # element-wise multiplication for Estimated number of people (healthy & sick) in the population per age per gender
    
    N = population_size-num_already_diagnosed # The entire population that is not scanned yet
    n = min(screening_pop,N)
    
    mean_number_of_detected_per_age_per_stage = (n*D)/N #Expectation of HG distribution
    total_number_of_detected = (mean_number_of_detected_per_age_per_stage*proportion_of_sick_out_of_sick)+already_diagnosed # This will return the new detected % in each group - will be the same as there is no priority between groups
    new_prop_diagnosed = (total_number_of_detected/pai)[0][0,0]  #the new % of detected will be the same for each stage so will take one of them

    return new_prop_diagnosed

In [17]:
def create_pai0 (dicti,screening_pop,treatment_policy):
    
    # We save each calculation of this function in a dictionary - "pai0_dict". If we've already calculated then use the saved results, else calculate from scratch
    key = (screening_pop,treatment_policy)
    
    if key in pai0_dict:
        return pai0_dict[key]
    
    else:   
        num_age_groups = dicti['num_age_groups']
        num_disease_stages = dicti['num_disease_stages']
        new_prop_diagnosed = delta_scanning(dicti,screening_pop)
        efficacy = dicti['efficacy']
        pai=dicti['gl_pai']

        pai0_female=np.zeros((num_age_groups,len(stages)))
        pai0_male=np.zeros((num_age_groups,len(stages)))

        #Undiagonsed population
        num_of_states_not_discovered = len(states_not_discovered)

        pai0_female[:,:num_of_states_not_discovered]=pai[0][:,:num_of_states_not_discovered]*(1-new_prop_diagnosed)
        pai0_male[:,:num_of_states_not_discovered]=pai[1][:,:num_of_states_not_discovered]*(1-new_prop_diagnosed)

        #Diagnosed population
        if treatment_policy[0] == -1: #"Easy" scenario where there is no treatment - we just diagnose with the normal proportion & no treatment 
            # Update the states where f=d, meaning we diagnosed people in each state and then 
            for state in states_where_f_equals_d:
                state_pai_index = pai_indices_dict[state]
                state_stage = state[0]
                pai0_female[:,state_pai_index]=pai[0][:,state[0]]*new_prop_diagnosed
                pai0_male[:,state_pai_index]=pai[1][:,state[0]]*new_prop_diagnosed

        else: #If there is treatment - it depends on the stage & age in policy
            for age in range(num_age_groups):
                for stage in range(config['num_stages']):
                    if (stage>=treatment_policy[0] and age<=treatment_policy[1]) or (treatment_policy[0]>=0 and stage>treatment_policy[0]): #if we're in policy - initial population can transition either to success or the corresponding unsuccesful treatment
                        
                        #unsuccessful treatment
                        unsuccesfully_treated_corresponding_stage = ('u'+str(stage))
                        unsuccesfully_treated_corresponding_stage_index = pai_indices_dict[unsuccesfully_treated_corresponding_stage]
                        pai0_female[age,unsuccesfully_treated_corresponding_stage_index] = pai[0][age,stage]*new_prop_diagnosed*(1-efficacy[stage])
                        pai0_male[age,unsuccesfully_treated_corresponding_stage_index] = pai[1][age,stage]*new_prop_diagnosed*(1-efficacy[stage])

                        #succesful treatment
                        succesfully_treated_corresponding_stage_index = pai_indices_dict['success']
                        pai0_female[age,succesfully_treated_corresponding_stage_index] += pai[0][age,stage]*new_prop_diagnosed*efficacy[stage]
                        pai0_male[age,succesfully_treated_corresponding_stage_index] += pai[1][age,stage]*new_prop_diagnosed*efficacy[stage]

                    else: #not in policy - we're diagnosing people as the prior
                        discovered_stage=(stage,stage)
                        discovered_pai_index = pai_indices_dict[discovered_stage]
                        pai0_female[age,discovered_pai_index]=pai[0][age,stage]*new_prop_diagnosed
                        pai0_male[age,discovered_pai_index]=pai[1][age,stage]*new_prop_diagnosed

        # Change the matrices into a vector - to actually be pai_0 
        pai0_female=pai0_female.reshape(pai0_female.size, order="F")
    #         pai0_female = pai0_female/pai0_female.sum() #if normalization if needed
        pai0_male=pai0_male.reshape(pai0_male.size, order="F")
    #         pai0_male = pai0_male/pai0_male.sum() #if normalization if needed
        matrix_cols_shape = len(transient_indices_dict)*num_age_groups+len(absorbing_indices_dict)
        pai0=np.array((pai0_female[:matrix_cols_shape],pai0_male[:matrix_cols_shape]))

        # Save results for better run time if needed again (no reduntant calculations)
        pai0_dict[screening_pop,treatment_policy] = pai0

        return pai0  

## Life Expectancy

In [18]:
def life_expectancy(config,pai0,gender,N):
    '''equals to pai0 * the sum of each row in the N matrix divided by the number of steps'''
    step = config['step']
    size=len(N)
    life_ex=np.array([0]*size)
    for line in range(size):
        life_ex[line]=N[line,:].sum()
    pai0_transient = pai0[gender][:size] #pai0[0:size] represents the initial vector for the transient states. pai0[size:] represents the initial vector for the absorbing states
    total_LE=np.dot(pai0_transient,life_ex)/step
    
    return total_LE

## Sum_checkups

In [19]:
def sum_checkups(pai0,gender,N,yi):
    size= len(N)
    total_time_stage=np.array([0]*size)
    total_sum=0
    pai0_transient = pai0[gender][:size] #pai0[0:size] represents the initial vector for the transient states. pai0[size:] represents the initial vector for the absorbing states

    for key in transient_indices_dict:
        if key[1] in (-1,0,1,2): #only if the individual is not treated nor diagnosed we calculate the checkup amounts.
            for col in transient_indices_dict[key]:
                total_time_stage[col]=np.dot(pai0_transient,N[:,col]) 
            # The total life expectancy in days of all the diagnosed individuals
                total_sum+=total_time_stage[col].sum() * yi[key[1]]

    return total_sum

## Screening Costs

The next step is to describe the country’s total costs of screening, C(q), as a function of the screening population’s size, q. Population screening is based on the examination of the undiagnosed population. As q increases, it becomes more difficult to locate and convince people to undergo screening. We chose the isoelastic function, C(q)=$c_s∙\frac{\partial(q^{1-a})}{1-a}$ - 

$c_s$ represents the fixed cost for a single HCV examination (i.e. screening) and a is a parameter that reflects the change in the willingness of the population to be screened. Note that if a=0, C(q)=c_s q, leading to a linear cost of screening.

In [ ]:
def screening_costs(dicti,p,cost_efficient):    
    s_screen = dicti['s_screen']
    power= (1/(1-cost_efficient))
    screening_cost=((1-cost_efficient)*s_screen*p)**power 
#     screening_cost = max(s_screen*((p**(1-cost_efficient))/(1-cost_efficient)),0)
    return screening_cost

## Calculate Utility

In [ ]:
def calc_utility (dicti,treatment_policy, yi,scanned_pop,cost_efficient,treatment_cost,v):
    
    global utility_dict
    
    # We save each calculation of this function in a dictionary - "pai0_dict". If we've already calculated then use the saved results, else calculate from scratch
    key = (treatment_policy, yi,scanned_pop,treatment_cost,v,cost_efficient)
    
    if key in utility_dict:
        return utility_dict[key]
    
    genders = dicti['genders']
    scanned_pop = scanned_pop
    screening_cost = screening_costs(dicti,scanned_pop,cost_efficient)
    s_m = dicti['s_m'] # cost of mortality
    s_c=dicti['s_c'] # cost of checkups
    treatment_cost = treatment_cost   
    for gender in genders:
          utility_vars_dict[gender] = calc_utility_vars (dicti,treatment_policy,gender, yi,scanned_pop,treatment_cost)

    LE,total_checkups,p,hcv_death_count = np.array(list(utility_vars_dict.values())).sum(axis=0)

    utility = LE*v - (screening_cost + p*treatment_cost+ hcv_death_count*s_m + total_checkups*s_c)

    return utility,hcv_death_count,p,total_checkups,LE

In [ ]:
def calc_utility_vars (dicti,treatment_policy,gender, yi,scanned_pop,treatment_cost):
    
    n,b = N_and_B(dicti,treatment_policy,gender, yi)
    size= len(n)
    pai0 = create_pai0 (dicti,scanned_pop,treatment_policy)
    pai0_transient = pai0[:][:,:size] # pai0[:][:,:size][gender]==pai0[gender][:size] 
    LE = life_expectancy(dicti,pai0_transient,gender,n)
    total_paiB = np.dot(pai0_transient[gender][:size],b)
    total_checkups = sum_checkups(pai0_transient,gender,n,yi)
    p = total_paiB[0,2] + total_paiB[0,3] # The number of treated patients equals to the number of people that ended in m3 & m4
    hcv_death_count = total_paiB[0,1]+total_paiB[0,3] # People died in m2 or m4

    return [LE,total_checkups,p,hcv_death_count]

## Calculate Country Utility if no Treatment is applied

In [ ]:
def gvmt_noTreatment_utility (dicti,cost_efficient,v):
    treatment_policy=(-1,-1)
    treatment_cost = 0
    yi = (0.0005479452054794521, 0.0005479452054794521, 0.0005479452054794521, 0)
    scanned_pop = 0
    s_t=0
    

    if v == 40000: #GDP1
        min_G_utility,min_hcv_death_count,min_p,min_total_checkups,min_LE = calc_utility (config,treatment_policy, yi,0,cost_efficient,treatment_cost,v)
        print(f"utility = {min_G_utility}")
        print (f"hcv_death_count = {min_hcv_death_count}")
        print (f"p = {min_p}")
        print (f"total_checkups = {min_total_checkups}")
        print (f"LE = {min_LE}")
        print ('---------------------')
    else:
        min_G_utility,min_hcv_death_count,min_p,min_total_checkups,min_LE = calc_utility (config,treatment_policy, yi,0,cost_efficient,treatment_cost,v)
        print(f"min utility = {min_G_utility}")
        print (f"min hcv_death_count = {min_hcv_death_count}")
        print (f"min p = {min_p}")
        print (f"min total_checkups = {min_total_checkups}")
        print (f"min LE = {min_LE}")
        print ('---------------------')
        
    return min_G_utility, min_LE

## Optimal Policy

In [ ]:
def opt_policy(dicti,scanned_pop,cost_efficient,treatment_cost,v):
    
    # We save each calculation of this function in a dictionary - "opt_policy_dict". If we've already calculated then use the saved results, else calculate from scratch
    key = (scanned_pop,cost_efficient,cost_efficient,treatment_cost,v)
    
    if key in opt_policy_dict:
        return opt_policy_dict[key]
    
    max_utility=0
    max_policy=None
    max_xi=None
    max_hcv_death=None
    max_LE=None

    scanned_pop = scanned_pop
    policies = dicti['policies']
    yi_all = dicti['yi_all']

    #    for policy in tqdm(policies): #65
    for policy in policies: #65     
        for yi in yi_all:
            utility,hcv_death_count,p,total_checkups,LE = calc_utility(dicti,policy, yi,scanned_pop,cost_efficient,treatment_cost,v)
            if utility>max_utility:
                max_utility=utility
                max_policy=policy
                max_yi=yi
                max_p=p
                max_hcv_death=hcv_death_count
                max_LE=LE
    
    # Save results for better run time if needed again (no reduntant calculations)
    opt_policy_dict[scanned_pop,cost_efficient,cost_efficient,treatment_cost,v] = (max_utility, max_policy, max_yi, max_p, max_hcv_death, max_LE) 


    return (max_utility, max_policy, max_yi, max_p, max_hcv_death, max_LE) 

In [ ]:
opt_policy_dict = {}

# Run Scenarios

## Basic Model + One-time Screening campaign 

In [ ]:
# Save results into Dataframe
results_df = pd.DataFrame(columns=['treatment_cost','policy_stage','policy_age','scanned_pop','cost_efficient','num_treated','HCV_death_cases','LE_added','v','G_utility','PC_utility','Gov_gain','F0_freq_annual','F1_freq_annual','F2_freq_annual'])

In [ ]:
def scenarios_1(dicti,scanned_pop,cost_efficient):
    for v in dicti['v']:
        print('')
        print('*********')
        print(f"v = {v}")
        print('*********')
        graph_stopping_counter = 0 # Its purpose is to continue plotting the graph for two more points     
       
        min_G_utility,min_LE = gvmt_noTreatment_utility (dicti,cost_efficient,v)

        # Create checkup vector (yi)
        dicti['yi_options'] = np.array([2,1,2/3,1/2,1/3,1/4,1/5])/dicti['step']  # amount of checkups in a year
#         dicti['yi_options'] = np.array([2,1,1/5,1/4])/dicti['step']  # amount of checkups in a year
        yi_all=list(itertools.product(dicti['yi_options'],dicti['yi_options'],dicti['yi_options'],range(1)))
        for i in range(len(yi_all)-1,-1,-1):
            if yi_all[i][0]>yi_all[i][1] or yi_all[i][1]>yi_all[i][2]:
                yi_all.pop(i)
        dicti['yi_all']=yi_all    
        for treatment_cost in config['total_s_t']:
            print(f"treatment_cost = {treatment_cost}")

            max_utility, max_policy, max_yi, max_p, max_hcv_death, max_LE = opt_policy (config,scanned_pop,cost_efficient,treatment_cost,v)
            print(f"max_yi = {max_yi}")
            Gov_gain = max_utility-min_G_utility
            print(f"Gov_gain = {Gov_gain}")
            if Gov_gain <= 0:
                graph_stopping_counter +=1
                print(f"INCREASED GRAPH_STOPPING_COUNTER = {graph_stopping_counter}")
                if graph_stopping_counter ==1:
                    break
            LE_added = max(max_LE-min_LE,0)
            max_yi_year= np.array(max_yi)*config['step']
            F0_freq_annual=max_yi_year[0]
            F1_freq_annual=max_yi_year[1]  
            F2_freq_annual=max_yi_year[2]

            print(f"max_yi_year = {max_yi_year}")
            # Update  yi_all  (if the last stage checkup becomes lower from a certain price (e.g. drops from 2 to 1), it won't be 2 again)
            # For better runtime
            for i in range(len(dicti['yi_all'])-1,-1,-1):
                f0=dicti['yi_all'][i][0]
                f1=dicti['yi_all'][i][2]
                f2=dicti['yi_all'][i][2]
                if (f0>=max_yi[0]*1.02) or (f1>=max_yi[1]*1.02) or (f2>=max_yi[2]*1.02):
                    dicti['yi_all'].pop(i)
            print('Length of yi_all_after:',len(dicti['yi_all']))

            PC_utility = treatment_cost*max_p

            print(f"max_policy = {max_policy}")
            print(f"P = {max_p}")
            print('')
            new_row_index = results_df.shape[0]+1
            results = [treatment_cost,max_policy[0],max_policy[1],scanned_pop,cost_efficient,max_p,max_hcv_death,LE_added,v,max_utility,PC_utility,Gov_gain,F0_freq_annual,F1_freq_annual,F2_freq_annual]
            results_df.loc[new_row_index,:]=results

Scenarios_2 consists of both Basic model (scanned_pop = 0) & Extension 1

In [ ]:
def scenarios_2(dicti):
    for cost_efficient in dicti['cost_efficient']:
        for scanned_pop in tqdm(dicti['optional_ps'],desc='Scanned_pop'): 
            print('')
            print(f"scanned_pop = {scanned_pop}")
            print(f"cost_efficient = {cost_efficient}")
            print('')            
            scenarios_1(dicti,scanned_pop,cost_efficient)

In [ ]:
scenarios_2(config)

## Extension 2 - Discount for Quantity

In [ ]:
def calc_utility_with_threshold (dicti,treatment_policy, yi,scanned_pop,cost_efficient,treatment_cost,v,treated_threshold):

    utility_vars_dict = {}
    genders = dicti['genders']
    scanned_pop = scanned_pop
    screening_cost = screening_costs(dicti,scanned_pop,cost_efficient)
    s_m = dicti['s_m'] # cost of mortality
    s_c=dicti['s_c'] # cost of checkups

    for gender in genders:
        utility_vars_dict[gender] = calc_utility_vars(dicti, treatment_policy, gender, yi, scanned_pop, treatment_cost)

    LE, total_checkups, p, hcv_death_count = np.array(list(utility_vars_dict.values())).sum(axis=0)

    # If there are more than p people that need to undergo treatment, we will charge only p of them (non-linear pricing)
    if p > treated_threshold:
        p_charged = treated_threshold
    else:
        p_charged = p
    pc_utility = p_charged*treatment_cost - screening_cost
    utility = LE * v - (p_charged * treatment_cost + hcv_death_count * s_m + total_checkups * s_c)

    return utility, hcv_death_count, p, total_checkups, LE, pc_utility


def opt_policy_threshold (dicti,scanned_pop,cost_efficient,treatment_cost,v,treated_threshold):

    max_utility=0
    max_policy=None
    max_yi=None
    max_hcv_death=None
    max_LE=None

    scanned_pop = scanned_pop
    policies = dicti['policies']
    yi_all = dicti['yi_all']

    #    for policy in tqdm(policies): #65
    for policy in policies: #65     
        for yi in yi_all:
            gov_utility,hcv_death_count,p,total_checkups,LE,PC_utility = calc_utility_with_threshold(dicti,policy, yi,scanned_pop,cost_efficient,treatment_cost,v,treated_threshold)
            if gov_utility>max_utility:
                max_utility=gov_utility
                max_pc_utility = PC_utility
                max_policy=policy
                max_yi=yi
                max_p=p
                max_hcv_death=hcv_death_count
                max_LE=LE

    return (max_utility, max_policy, max_yi, max_p, max_hcv_death, max_LE,max_pc_utility)

def scenarios_3(dicti,scanned_pop,cost_efficient,treated_threshold):
    res = []
    for v in dicti['v']:
    graph_stopping_counter = 0 # Its purpose is to continue plotting the graph for two more points
    min_G_utility,min_LE = gvmt_noTreatment_utility (dicti,cost_efficient,v)
    for treatment_cost in config['total_s_t']:
        max_utility, max_policy, max_yi, max_p, max_hcv_death, max_LE,max_pc_utility = opt_policy_threshold (dicti,scanned_pop,cost_efficient,treatment_cost,v,treated_threshold)
        Gov_gain = max_utility-min_G_utility
        if Gov_gain <= 0:
            graph_stopping_counter +=1
            if graph_stopping_counter >=2:
                break
        LE_added = max(max_LE-min_LE,0)
        max_yi_year= np.array(max_yi)*config['step']
        F0_freq_annual=max_yi_year[0]
        F1_freq_annual=max_yi_year[1]
        F2_freq_annual=max_yi_year[2]

        # Update  yi_all  (if the last stage checkup becomes lower from a certain price (e.g. drops from 2 to 1), it won't be 2 again)
        # For better runtime
        for i in range(len(dicti['yi_all']) - 1, -1, -1):
            f0 = dicti['yi_all'][i][0]
            f1 = dicti['yi_all'][i][2]
            f2 = dicti['yi_all'][i][2]
            if (f0 >= max_yi[0] * 1.02) or (f1 >= max_yi[1] * 1.02) or (f2 >= max_yi[2] * 1.02):
                dicti['yi_all'].pop(i)


        results = [treatment_cost,max_policy[0],max_policy[1],scanned_pop,cost_efficient,treated_threshold,max_p,max_hcv_death,LE_added,v,max_utility,max_pc_utility,Gov_gain,F0_freq_annual,F1_freq_annual,F2_freq_annual]
        res.append(results)
    rest = pd.DataFrame(res, columns=['treatment_cost', 'policy_stage', 'policy_age', 'scanned_pop', 'cost_efficient','treated_threshold',
                                  'num_treated', 'HCV_death_cases', 'LE_added', 'v', 'G_utility', 'PC_utility',
                                  'Gov_gain', 'F0_freq_annual', 'F1_freq_annual', 'F2_freq_annual'])
    return rest

## Extension 3 - Timed Screening Following Discount for Quantity

In [ ]:
opt_policy_dynamic_scanning_dict = {}

In [ ]:
results_df = pd.DataFrame(columns=['treatment_cost','policy_stage','policy_age','scanned_pop','cost_efficient','treated_threshold','num_treated_since_cutoff','num_treated_until_cutoff','HCV_death_cases','LE_added','v','G_utility','PC_utility','Gov_gain','F0_freq_annual','F1_freq_annual','F2_freq_annual','future_years'])

In [ ]:
# To Calculate the Life expectancy of people who died until the future point in time, we use regression
def LE_of_absorbed_people_regression(x_train,y_train,x_test):
    X_test = np.array([x_test])
    X = x_train.reshape(-1,1)
    y = y_train.reshape(-1,1)
    X_test=X_test.reshape(-1,1)
    
    # Degree=3 yielded the best results
    poly = PolynomialFeatures(degree=3)
    X_ = poly.fit_transform(X)
    X_test_ = poly.fit_transform(X_test)
    lg = LinearRegression()
    lg.fit(X_, y)
    return lg.predict(X_test_)[0][0]

def get_sample_data_for_regression(dicti, future_years,scanned_pop,treatment_policy,yi):
    genders = dicti['genders']
    pai0 = create_pai0 (dicti,scanned_pop,treatment_policy)
    for gender in genders:
        q,r = create_matrix(dicti,treatment_policy,gender, yi)
        # Build transition matrix in the future
        zero_matrix = np.zeros((r.shape[1],r.shape[0]))
        i_matrix = np.eye(r.shape[1])

        mat_bottom = np.concatenate((zero_matrix,i_matrix),axis=1)
        mat_top = np.concatenate(((q,r)),axis=1)
        mat = np.concatenate((mat_top,mat_bottom),axis=0)

        # probablity transition matrix in the future
        probablity_transition_matrix_future = np.linalg.matrix_power(mat,future_years*config['step'])

        if gender==0:
            pai_future_female = np.dot(pai0[gender],probablity_transition_matrix_future)
        if gender==1:
            pai_future_male = np.dot(pai0[gender],probablity_transition_matrix_future)

    # Get pai vector in the future
    pai_future = np.array((pai_future_female,pai_future_male))
    pop_size = pai0.sum()
    absorbed_diff = pai_future[:][:,304:].sum() - pai0[:][:,304:].sum()
    pop_alive = pop_size-absorbed_diff
    LE_cutoff_from_alive = pop_alive*future_years

    LE_diff=0
    for gender in genders:
        n,b = N_and_B(dicti,treatment_policy,gender, yi)
        LE_diff += life_expectancy(config,pai0,gender,n)-life_expectancy(config,pai_future,gender,n)

    LE_cutoff_from_absorbed = LE_diff-LE_cutoff_from_alive

    return (absorbed_diff,LE_cutoff_from_absorbed)


sample_data_for_regression_dict = {}
    
def sample_data(dicti, scanned_pop,treatment_policy):
    genders = dicti['genders']
# Removed yi to improve run time - affects accuracy just a tiny bit
    key = (scanned_pop,treatment_policy)
    
    if key in sample_data_for_regression_dict:
        return sample_data_for_regression_dict[key]
    
    else:
        x_train=np.array([])
        y_train=np.array([])

        for future_year in range(1,20):
            regression_results = get_sample_data_for_regression(dicti, future_year,scanned_pop,treatment_policy,yi)
            new_x = regression_results[0]
            new_y = regression_results[1]

            x_train = np.append(x_train,new_x)
            y_train = np.append(y_train,new_y) 
            
        sample_data_for_regression_dict[scanned_pop,treatment_policy] = (x_train,y_train) 
        
        return (x_train,y_train)

In [ ]:
def delta_scanning_dynamic_scanning (dicti,pai_future,screening_pop,future_years):
    '''Input: Pai vector + screening population + future_years (to determine total population size)
       Output: new Pai vector after scanning'''

    efficacy = dicti['efficacy']

    # Getting the population size - Based on CBS (Lamas) data. 
    # row 0 is 2012 data
    population_size = israel_population_size.population_size[future_years]

    # already diagnosed
    pai_future_transient = pai_future[:][:,:304]
    pai_sum = pai_future[:][:,:304].sum() # num of alive (after discounts & deaths)

    # Calculate not diagnosed yet population
    # Equivalent to pai_future[:][:,0:224].sum()
    not_diagnosed_yet = 0
    for state,index in transient_indices_dict.items():
        if isinstance(state,tuple):
            not_diagnosed_yet += (pai_future[:][:,index].sum())

    # Calculate already diagnosed
    num_already_diagnosed = 0
    for state,index in transient_indices_dict.items():
        if state in states_treated:
            num_already_diagnosed += (pai_future[:][:,index].sum())

    D = not_diagnosed_yet # Number of infected not diagnosed yet
    N = population_size-num_already_diagnosed
    n = min(screening_pop,N)

    proportion_of_sick_out_of_sick = pai_future[:][:,0:224]/pai_future[:][:,0:224].sum()

    mean_number_of_detected_per_age_per_stage = (n*D)/N #Expectation of HG distribution
    total_number_of_detected = (mean_number_of_detected_per_age_per_stage*proportion_of_sick_out_of_sick) # This will return the new detected % in each group - will be the same as there is no priority between groups
    # Update pai to include after scanning
    # From the not diagnosed population - move to diagnosed
    # pai_future[:][:,0:224] = pai_future[:][:,0:224]-total_number_of_detected

    # Iterate over transient stages                  
    for stage in transient_indices_dict:
        # But only the untreated ones
        if isinstance(stage,tuple):
            for row in indices_dict[stage]: 
                f_state=stage[0]
                d_state = stage[1] 
                age = indices_dict[stage].index(row)# get the age group of the current row
    
    #         If in policy:
                if (f_state>=treatment_policy[0] and age<=treatment_policy[1]) or (treatment_policy[0]>=0 and f_state>treatment_policy[0]): #if we're in policy -  population can transition either to success or the corresponding unsuccesful treatment
                    
                    # successful treatment
                    pai_future[:][:,indices_dict['success'][age]]+=total_number_of_detected[:][:,row]*efficacy[f_state]
                    pai_future[:][:,indices_dict[stage][age]]-=total_number_of_detected[:][:,row]*efficacy[f_state]
                    # unsuccessful
                    unsuccesfully_treated_corresponding_stage = ('u'+str(f_state))
                    pai_future[:][:,indices_dict[unsuccesfully_treated_corresponding_stage][age]]+=total_number_of_detected[:][:,row]*(1-efficacy[f_state])  
                    pai_future[:][:,indices_dict[stage][age]]-=total_number_of_detected[:][:,row]*(1-efficacy[f_state])  
                
                else: # Not in policy
                    discovered_stage=(stage[0],stage[0])
                    pai_future[:][:,indices_dict[discovered_stage][age]]+=total_number_of_detected[:][:,row]
                    pai_future[:][:,indices_dict[stage][age]]-=total_number_of_detected[:][:,row]

    return pai_future

In [ ]:
def calc_utility_dynamic_scanning (dicti,treatment_policy, yi,scanned_pop,cost_efficient,treatment_cost,v,future_years,treated_threshold):
    genders = dicti['genders']
    s_m = dicti['s_m'] # cost of mortality
    s_c=dicti['s_c'] # cost of checkups 
    yearly_interest = dicti['r']*365
    p_treated_future = 0
    # samples for regression (needed to calculate LE of absorbed people)

    # calculate treatment_cost future years from now
    # At first we scan zero people
    scanned_pop_initial=0
    pai0 = create_pai0 (dicti,scanned_pop_initial,treatment_policy)
    for gender in genders:
        
        #-------------------------------------------
        # get sample data for regression 
        x_train,y_train = sample_data(dicti, scanned_pop_initial,treatment_policy)   
        #-------------------------------------------
        # Back to utility calculation
        
        # first twenty years
        q,r = create_matrix(dicti,treatment_policy,gender, yi)

        # Build transition matrix in the future
        zero_matrix = np.zeros((r.shape[1],r.shape[0]))
        i_matrix = np.eye(r.shape[1])

        mat_bottom = np.concatenate((zero_matrix,i_matrix),axis=1)
        mat_top = np.concatenate(((q,r)),axis=1)
        mat = np.concatenate((mat_top,mat_bottom),axis=0)

        # probablity transition matrix in the future
        probablity_transition_matrix_future = np.linalg.matrix_power(mat,future_years*config['step'])

        
        if gender==0:
            pai_future_female = np.dot(pai0[gender],probablity_transition_matrix_future)
        if gender==1:
            pai_future_male = np.dot(pai0[gender],probablity_transition_matrix_future)
    
    # Combine both female & male pai vectors
    pai_future = np.array((pai_future_female,pai_future_male))
    
        
    # Calculate life_expectancy until the future "cut-off" point
    pop_size = pai0.sum()
    absorbed_diff = pai_future[:][:,304:].sum() - pai0[:][:,304:].sum()
    pop_alive = pop_size-absorbed_diff
    LE_cutoff_from_alive = pop_alive*future_years
    LE_cutoff_from_absorbed = LE_of_absorbed_people_regression(x_train,y_train,absorbed_diff)

    # update parameters for future regression
    x_train = np.append(x_train,absorbed_diff)
    y_train = np.append(y_train,LE_cutoff_from_absorbed)

    LE_cutoff = LE_cutoff_from_alive+LE_cutoff_from_absorbed


    p_treated_future = pai_future[:][:,[306,307]].sum()
    hcv_death_count_until_cutoff = pai_future[:][:,[305,307]].sum()
    
    # Run from point in the future to infinity (the new "pai0" is the point in the future)    
    # Calculate utility from cutoff to the end
    pai_future_after_scanning =  delta_scanning_dynamic_scanning (dicti,pai_future,scanned_pop,future_years)
    for gender in genders:
        utility_vars_dict[gender] = calc_utility_vars_dynamic_scanning (dicti,treatment_policy,gender, yi,scanned_pop,treatment_cost,pai_future_after_scanning)
                
    checkups_until_cutoff = sum_checkups_dynamic_scanning(pai_future,gender,yi,future_years)
    LE,checkups_from_cutoff,p,hcv_death_count_from_cutoff = np.array(list(utility_vars_dict.values())).sum(axis=0)
    total_checkups = checkups_until_cutoff+checkups_from_cutoff
    total_LE = LE_cutoff + LE
    
    # If there are more than p people that need to undergo treatment, we will charge only p of them (non-linear pricing)
    if p > treated_threshold:
        p_charged = treated_threshold
    else:
        p_charged = p
    
    screening_cost = screening_costs(dicti,scanned_pop,cost_efficient)
    # As the screening happens in the future - we need to calculate its present value
    screening_cost_pv = screening_cost/(1+yearly_interest)**future_years
      
    PC_utility = (p_treated_future+p_charged)*treatment_cost
    total_hcv_death = hcv_death_count_until_cutoff+hcv_death_count_from_cutoff
    
    utility = total_LE*v - (screening_cost_pv + (p_treated_future+p_charged)*treatment_cost + total_hcv_death*s_m + total_checkups*s_c)
    
    return utility,total_hcv_death,p,p_treated_future,total_checkups,total_LE,PC_utility

In [ ]:
def calc_utility_vars_dynamic_scanning (dicti,treatment_policy,gender, yi,scanned_pop,treatment_cost,pai_future):
    
    n,b = N_and_B(dicti,treatment_policy,gender, yi)
    size= len(n)
    pai0_transient = pai_future[:][:,:size] # pai0[:][:,:size][gender]==pai0[gender][:size] 
    LE = life_expectancy(dicti,pai0_transient,gender,n)
    total_paiB = np.dot(pai0_transient[gender][:size],b)
    total_checkups = sum_checkups(pai0_transient,gender,n,yi)
    p = total_paiB[0,2] + total_paiB[0,3] # The number of treated patients equals to the number of people that ended in m3 & m4
    hcv_death_count = total_paiB[0,1]+total_paiB[0,3] # People died in m2 or m4

    return [LE,total_checkups,p,hcv_death_count]

In [ ]:
def opt_policy_dynamic_scanning(dicti,scanned_pop,cost_efficient,treatment_cost,v,future_years,treated_threshold):
    
    # We save each calculation of this function in a dictionary - "opt_policy_dict". If we've already calculated then use the saved results, else calculate from scratch
    key = (scanned_pop,cost_efficient,treatment_cost,v,future_years,treated_threshold)
    
    if key in opt_policy_dynamic_scanning_dict:
        return opt_policy_dynamic_scanning_dict[key]
    
    max_utility=0
    max_policy=None
    max_xi=None
    max_hcv_death=None
    max_LE=None

    policies = dicti['policies']
    yi_all = dicti['yi_all']

    #    for policy in tqdm(policies): #65
    for policy in policies: #65     
        for yi in yi_all:
            utility,hcv_death_count,p,p_treated_future,total_checkups,LE,PC_utility = calc_utility_dynamic_scanning (dicti,policy, yi,scanned_pop,cost_efficient,treatment_cost,v,future_years,treated_threshold)
            if utility>max_utility:
                max_utility=utility
                max_policy=policy
                
                max_yi=yi
                max_p=p
                max_p_treated_future = p_treated_future
                max_hcv_death=hcv_death_count
                max_LE=LE
                max_pc_utility = PC_utility
    # Save results for better run time if needed again (no reduntant calculations)
    opt_policy_dynamic_scanning_dict[scanned_pop,cost_efficient,treatment_cost,v,future_years,treated_threshold] = (max_utility, max_policy, max_yi, max_p, max_p_treated_future, max_hcv_death, max_LE,max_pc_utility) 


    return max_utility, max_policy, max_yi, max_p, max_p_treated_future, max_hcv_death, max_LE,max_pc_utility

In [ ]:
def sum_checkups_dynamic_scanning(pai_future,gender,yi,future_years):
    total_sum=0
    for key in transient_indices_dict:
        if key[1] in (-1,0,1,2): #only if the individual is not treated nor diagnosed we calculate the checkup amounts.
            for col in transient_indices_dict[key]:
                # The total life expectancy in days of all the diagnosed individuals
                total_sum+= pai_future[gender][col].sum() * yi[key[1]] * 365 * future_years

    return total_sum

In [ ]:
def scenarios_5_timed_screening(dicti,future_years):
    cost_efficient=0
    for v in tqdm(dicti['v'],desc='v'):
        print('') 
        print(f"v = {v}")
        print(datetime.datetime.now())
        min_G_utility,min_LE = gvmt_noTreatment_utility (dicti,cost_efficient,v)
        for scanned_pop in tqdm(dicti['optional_ps'],desc='Scanned_pop'): 
            print(f"scanned_pop = {scanned_pop}")
            print('')                      
            for threshold in dicti['op_threshs']:
                graph_stopping_counter = 0 # Its purpose is to continue plotting the graph for two more points 
                
                # Create checkup vector (yi)
                dicti['yi_options'] =np.array([2,1,2/3,1/2,1/3,1/4,1/5])/config['step']  # amount of checkups in a year
                yi_all=list(itertools.product(dicti['yi_options'],dicti['yi_options'],dicti['yi_options'],range(1)))
                for i in range(len(yi_all)-1,-1,-1):
                    if yi_all[i][0]>yi_all[i][1] or yi_all[i][1]>yi_all[i][2]:
                        yi_all.pop(i)
                dicti['yi_all']=yi_all
                
                for treatment_cost in dicti['total_s_t']:
                    print(f"threshold = {threshold}")
                    print(f"treatment_cost = {treatment_cost}")
                    max_utility, max_policy, max_yi, max_p,max_p_treated_future, max_hcv_death, max_LE,max_pc_utility = opt_policy_dynamic_scanning(dicti,scanned_pop,cost_efficient,treatment_cost,v,future_years,threshold)
                    print('max_LE',(max_LE))
                    Gov_gain = max_utility-min_G_utility
                    print(f"Gov_gain = {Gov_gain}")
                    print(f"max_pc_utility = {max_pc_utility}")
                    print(f"max_hcv_death = {max_hcv_death}")
                    LE_added = max(max_LE-min_LE,0)
                    max_yi_year= np.array(max_yi)*config['step']
                    F0_freq_annual=max_yi_year[0]
                    F1_freq_annual=max_yi_year[1]  
                    F2_freq_annual=max_yi_year[2]
                    if Gov_gain <= 0:
                        
                        graph_stopping_counter +=1
                        print(f"INCREASED GRAPH_STOPPING_COUNTER = {graph_stopping_counter}")
                        if graph_stopping_counter >=1:
                            Gov_gain=0
                            new_row_index = results_df.shape[0]+1
                            results = [treatment_cost,max_policy[0],max_policy[1],scanned_pop,cost_efficient,threshold,max_p,max_p_treated_future,max_hcv_death,LE_added,v,max_utility,max_pc_utility,Gov_gain,F0_freq_annual,F1_freq_annual,F2_freq_annual,future_years]
                            results_df.loc[new_row_index,:]=results
                            break
                    
                    print ('LE_added',LE_added)
                    print(f"max_yi_year = {max_yi_year}")
                    # Update  yi_all  (if the last stage checkup becomes lower from a certain price (e.g. drops from 2 to 1), it won't be 2 again)
                    # For better runtime
#                     print('Length of yi_all_before:',len(dicti['yi_all']))
                    for i in range(len(dicti['yi_all'])-1,-1,-1):
                        f0=dicti['yi_all'][i][0]
                        f1=dicti['yi_all'][i][2]
                        f2=dicti['yi_all'][i][2]
#                         print('i',f0*365,f1*365,f2*365)
                        if (f0>=max_yi[0]*1.02) or (f1>=max_yi[1]*1.02) or (f2>=max_yi[2]*1.02):
#                             print('max_yi',max_yi[0]*1.02*365,max_yi[1]*1.02*365,max_yi[2]*1.02*365)
# #                             print('Removing:',tuple([z * 365 for z in i]))
#                             print('REMOVING:',f0*365,f1*365,f2*365 )
#                             index_of_i = dicti['yi_all'].index(i)
                            dicti['yi_all'].pop(i)
                    print('Length of yi_all_after:',len(dicti['yi_all']))
                    print(f"max_policy = {max_policy}")
                    print(f"P_until_cutoff = {max_p_treated_future}")
                    print(f"P_from_cutoff = {max_p}")
                    print('')
                    new_row_index = results_df.shape[0]+1
                    results = [treatment_cost,max_policy[0],max_policy[1],scanned_pop,cost_efficient,threshold,max_p,max_p_treated_future,max_hcv_death,LE_added,v,max_utility,max_pc_utility,Gov_gain,F0_freq_annual,F1_freq_annual,F2_freq_annual,future_years]
                    results_df.loc[new_row_index,:]=results

In [ ]:
future_years = list(range(0, 15, 1)) # Options for future years to perform screening
print(datetime.datetime.now())
scenarios_5_timed_screening(config,future_years)

## Extension 4 - Patent Expiry

In [ ]:
results_df = pd.DataFrame(columns=['treatment_cost','policy_stage','policy_age','scanned_pop','cost_efficient','num_treated_since_cutoff','num_treated_until_cutoff','HCV_death_cases','LE_added','v','G_utility','PC_utility','PC_utility_future_including_from_cutoff','Gov_gain','F0_freq_annual','F1_freq_annual','F2_freq_annual','future_years'])

In [ ]:
def calc_utility_expiry_date (dicti,treatment_policy, yi,scanned_pop,cost_efficient,treatment_cost,v,future_years):
    genders = dicti['genders']
    screening_cost = screening_costs(dicti,scanned_pop,cost_efficient)
    s_m = dicti['s_m'] # cost of mortality
    s_c=dicti['s_c'] # cost of checkups 
    p_treated_future = 0
    # samples for regression (needed to calculate LE of absorbed people)

    # calculate treatment_cost future years from now
    pai0 = create_pai0 (dicti,scanned_pop,treatment_policy)
    for gender in genders:
        
        #-------------------------------------------
        # get sample data for regression 
        x_train,y_train = sample_data(dicti, scanned_pop,treatment_policy)   
        #-------------------------------------------
        # Back to utility calculation
        
        # first twenty years
        q,r = create_matrix(dicti,treatment_policy,gender, yi)

        # Build transition matrix in the future
        zero_matrix = np.zeros((r.shape[1],r.shape[0]))
        i_matrix = np.eye(r.shape[1])

        mat_bottom = np.concatenate((zero_matrix,i_matrix),axis=1)
        mat_top = np.concatenate(((q,r)),axis=1)
        mat = np.concatenate((mat_top,mat_bottom),axis=0)

        # probablity transition matrix in the future
        probablity_transition_matrix_future = np.linalg.matrix_power(mat,future_years*config['step'])

        
        if gender==0:
            pai_future_female = np.dot(pai0[gender],probablity_transition_matrix_future)
        if gender==1:
            pai_future_male = np.dot(pai0[gender],probablity_transition_matrix_future)
    
    # Combine both female & male pai vectors
    pai_future = np.array((pai_future_female,pai_future_male))
    
        
    # Calculate life_expectancy until the future "cut-off" point
    pop_size = pai0.sum()
    absorbed_diff = pai_future[:][:,304:].sum() - pai0[:][:,304:].sum()
    pop_alive = pop_size-absorbed_diff
    LE_cutoff_from_alive = pop_alive*future_years
    LE_cutoff_from_absorbed = LE_of_absorbed_people_regression(x_train,y_train,absorbed_diff)

    # update parameters for future regression
    x_train = np.append(x_train,absorbed_diff)
    y_train = np.append(y_train,LE_cutoff_from_absorbed)

    LE_cutoff = LE_cutoff_from_alive+LE_cutoff_from_absorbed
        # Run from point in the future to infinity (the new "pai0" is the point in the future)
        # Assuming policy is changed to (0,15) as treatment price becomes generic

    LE_cutoff = LE_cutoff_from_alive+LE_cutoff_from_absorbed

    p_treated_future = pai_future[:][:,[306,307]].sum()
    hcv_death_count_until_cutoff = pai_future[:][:,[305,307]].sum()
    
    # Run from point in the future to infinity (the new "pai0" is the point in the future)    
    # Calculate utility from cutoff to the end
    for gender in genders:
        future_policy = (0,15)
        utility_vars_dict[gender] = calc_utility_vars_expiry_date (dicti,future_policy,gender, yi,scanned_pop,treatment_cost,pai_future)
                     
    checkups_until_cutoff = sum_checkups_expiry_date(pai_future,gender,yi,future_years)
    LE,checkups_from_cutoff,p,hcv_death_count_from_cutoff = np.array(list(utility_vars_dict.values())).sum(axis=0)
    total_checkups = checkups_until_cutoff+checkups_from_cutoff
        
    total_LE = LE_cutoff + LE
    total_hcv_death = hcv_death_count_until_cutoff+hcv_death_count_from_cutoff
    
    utility = total_LE*v - (screening_cost + p_treated_future*treatment_cost + p*treatment_cost*0.3 + total_hcv_death*s_m + total_checkups*s_c)
    return utility,total_hcv_death,p,p_treated_future,total_checkups,total_LE

In [ ]:
def opt_policy_expiry_date(dicti,scanned_pop,cost_efficient,treatment_cost,v,future_years):
    
    # We save each calculation of this function in a dictionary - "opt_policy_dict". If we've already calculated then use the saved results, else calculate from scratch
    key = (scanned_pop,cost_efficient,treatment_cost,v,future_years)
    
    if key in opt_policy_expiry_date_dict:
        return opt_policy_expiry_date_dict[key]
    
    max_utility=0
    max_policy=None
    max_xi=None
    max_hcv_death=None
    max_LE=None

    policies = dicti['policies']
    yi_all = dicti['yi_all']

    #    for policy in tqdm(policies): #65
    for policy in policies: #65     
        for yi in yi_all:
            utility,hcv_death_count,p,p_treated_future,total_checkups,LE = calc_utility_expiry_date(dicti,policy, yi,scanned_pop,cost_efficient,treatment_cost,v,future_years)
            if utility>max_utility:
                max_utility=utility
                max_policy=policy
                max_yi=yi
                max_p=p
                max_p_treated_future = p_treated_future
                max_hcv_death=hcv_death_count
                max_LE=LE
    
    # Save results for better run time if needed again (no reduntant calculations)
    opt_policy_expiry_date_dict[scanned_pop,cost_efficient,treatment_cost,v,future_years] = (max_utility, max_policy, max_yi, max_p,max_p_treated_future, max_hcv_death, max_LE) 


    return (max_utility, max_policy, max_yi, max_p, max_p_treated_future, max_hcv_death, max_LE) 

In [ ]:
def calc_utility_vars_expiry_date (dicti,treatment_policy,gender, yi,scanned_pop,treatment_cost,pai_future):
    
    n,b = N_and_B(dicti,treatment_policy,gender, yi)
    size= len(n)
    pai0_transient = pai_future[:][:,:size] # pai0[:][:,:size][gender]==pai0[gender][:size] 
    LE = life_expectancy(dicti,pai0_transient,gender,n)
    total_paiB = np.dot(pai0_transient[gender][:size],b)
    total_checkups = sum_checkups(pai0_transient,gender,n,yi)
    p = total_paiB[0,2] + total_paiB[0,3] # The number of treated patients equals to the number of people that ended in m3 & m4
    hcv_death_count = total_paiB[0,1]+total_paiB[0,3] # People died in m2 or m4

    return [LE,total_checkups,p,hcv_death_count]

In [ ]:
def sum_checkups_expiry_date(pai_future,gender,yi,future_years):
    total_sum=0
    for key in transient_indices_dict:
        if key[1] in (-1,0,1,2): #only if the individual is not treated nor diagnosed we calculate the checkup amounts.
            for col in transient_indices_dict[key]:
                # The total life expectancy in days of all the diagnosed individuals
                total_sum+= pai_future[gender][col].sum() * yi[key[1]] * 365 * future_years

    return total_sum

In [ ]:
def scenarios_4_expiry_date(dicti,future_years):
    cost_efficient=0
    for v in tqdm(dicti['v'],desc='v'):
        min_G_utility,min_LE = gvmt_noTreatment_utility (dicti,cost_efficient,v)
        for scanned_pop in tqdm(dicti['optional_ps'],desc='Scanned_pop'): 
            print('') 
            print(datetime.datetime.now())
            print(f"v = {v}")
            print(f"scanned_pop = {scanned_pop}")
            print('')            
            graph_stopping_counter = 0 # Its purpose is to continue plotting the graph for two more points            
        
        # Create checkup vector (yi)
            dicti['yi_options'] = np.array([2,1,2/3,1/2,1/3,1/4,1/5])/dicti['step']  # amount of checkups in a year
#             dicti['yi_options'] = np.array([2,1,1/5])/dicti['step']  # amount of checkups in a year
            yi_all=list(itertools.product(dicti['yi_options'],dicti['yi_options'],dicti['yi_options'],range(1)))
            for i in range(len(yi_all)-1,-1,-1):
                if yi_all[i][0]>yi_all[i][1] or yi_all[i][1]>yi_all[i][2]:
                    yi_all.pop(i)
            dicti['yi_all']=yi_all
            
            for treatment_cost in config['total_s_t']:
                print(f"treatment_cost = {treatment_cost}")
                max_utility, max_policy, max_yi, max_p,max_p_treated_future, max_hcv_death, max_LE = opt_policy_expiry_date (dicti,scanned_pop,cost_efficient,treatment_cost,v,future_years)
                print(f"max_yi = {max_yi}")
                Gov_gain = max_utility-min_G_utility
                print(f"Gov_gain = {Gov_gain}")
                if Gov_gain <= 0:
                    graph_stopping_counter +=1
                    print(f"INCREASED GRAPH_STOPPING_COUNTER = {graph_stopping_counter}")
                    if graph_stopping_counter >=1:
                        Gov_gain=0
                        LE_added = max(max_LE-min_LE,0)
                        max_yi_year= np.array(max_yi)*config['step']
                        F0_freq_annual=max_yi_year[0]
                        F1_freq_annual=max_yi_year[1]  
                        F2_freq_annual=max_yi_year[2]

                        PC_utility_future = treatment_cost*max_p_treated_future
                
                        PC_utility_future_including_from_cutoff = max_p_treated_future*treatment_cost + max_p*treatment_cost*0.3
                        new_row_index = results_df.shape[0]+1
                        results = [treatment_cost,max_policy[0],max_policy[1],scanned_pop,cost_efficient,max_p,max_p_treated_future,max_hcv_death,LE_added,v,max_utility,PC_utility_future,PC_utility_future_including_from_cutoff,Gov_gain,F0_freq_annual,F1_freq_annual,F2_freq_annual,future_years]
                        results_df.loc[new_row_index,:]=results
                        break
                LE_added = max(max_LE-min_LE,0)
                max_yi_year= np.array(max_yi)*config['step']
                F0_freq_annual=max_yi_year[0]
                F1_freq_annual=max_yi_year[1]  
                F2_freq_annual=max_yi_year[2]
                
                print(f"max_yi_year = {max_yi_year}")
                # Update  yi_all  (if the last stage checkup becomes lower from a certain price (e.g. drops from 2 to 1), it won't be 2 again)
                # For better runtime
                for i in range(len(dicti['yi_all'])-1,-1,-1):
                    f0=dicti['yi_all'][i][0]
                    f1=dicti['yi_all'][i][2]
                    f2=dicti['yi_all'][i][2]
                    if (f0>=max_yi[0]*1.02) or (f1>=max_yi[1]*1.02) or (f2>=max_yi[2]*1.02):
                        dicti['yi_all'].pop(i)
                print('Length of yi_all_after:',len(dicti['yi_all']))
                
                PC_utility_future = treatment_cost*max_p_treated_future
                
                PC_utility_future_including_from_cutoff = max_p_treated_future*treatment_cost + max_p*treatment_cost*0.3
                
                print(f"max_policy = {max_policy}")
                print(f"P_until_cutoff = {max_p_treated_future}")
                print(f"P_from_cutoff = {max_p}")
                print('')
                new_row_index = results_df.shape[0]+1
                results = [treatment_cost,max_policy[0],max_policy[1],scanned_pop,cost_efficient,max_p,max_p_treated_future,max_hcv_death,LE_added,v,max_utility,PC_utility_future,PC_utility_future_including_from_cutoff,Gov_gain,F0_freq_annual,F1_freq_annual,F2_freq_annual,future_years]
                results_df.loc[new_row_index,:]=results

In [ ]:
scenarios_4_expiry_date(config,future_years)

------------------------